In [ ]:
# ============================================
# 🔧 SETUP
# ============================================
!pip install -q pydub torchaudio transformers scikit-learn
!apt-get -y install ffmpeg
!pip install datasets peft accelerate
!pip install speechbrain
!pip install mir_eval pesq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove

In [ ]:
import os
import numpy as np
import torch
from pydub import AudioSegment
from sklearn.metrics import roc_curve
from torch.nn.functional import cosine_similarity
from transformers import AutoModel, AutoProcessor
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset
from pathlib import Path
import random
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
import torch.nn as nn
import torch.nn.functional as F
import math
import glob
import torchaudio
import soundfile as sf
import pandas as pd
from pesq import pesq
from mir_eval.separation import bss_eval_sources
from speechbrain.pretrained import SepformerSeparation as separator
from scipy.io.wavfile import write
import csv

<ipython-input-2-b90d5fac96e2>:24: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import SepformerSeparation as separator


In [ ]:

# Initialize SepFormer
sepformer = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir="tmpdir_sepformer")
sepformer.eval()

hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

SepformerSeparation(
  (mods): ModuleDict(
    (encoder): Encoder(
      (conv1d): Conv1d(1, 256, kernel_size=(16,), stride=(8,), bias=False)
    )
    (decoder): Decoder(256, 1, kernel_size=(16,), stride=(8,), bias=False)
    (masknet): Dual_Path_Model(
      (norm): GroupNorm(1, 256, eps=1e-08, affine=True)
      (conv1d): Conv1d(256, 256, kernel_size=(1,), stride=(1,), bias=False)
      (dual_mdl): ModuleList(
        (0-1): 2 x Dual_Computation_Block(
          (intra_mdl): SBTransformerBlock(
            (mdl): TransformerEncoder(
              (layers): ModuleList(
                (0-7): 8 x TransformerEncoderLayer(
                  (self_att): MultiheadAttention(
                    (att): MultiheadAttention(
                      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
                    )
                  )
                  (pos_ffn): PositionalwiseFeedForward(
                    (ffn): Sequential(
                      (0

In [ ]:
# ============================================
# 💾 Load Hugging Face Wav2Vec2 Model
# ============================================
model_name = "facebook/wav2vec2-base"
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModel.from_pretrained(model_name).to(device)
processor = AutoProcessor.from_pretrained(model_name) #tokenizer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
model.eval()

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

In [ ]:
# ✅ STEP 3: Define FiLM + Enhancement Model
class FiLM(torch.nn.Module):
    def __init__(self, in_channels, embedding_dim):
        super(FiLM, self).__init__()
        self.gamma = torch.nn.Linear(embedding_dim, in_channels)
        self.beta = torch.nn.Linear(embedding_dim, in_channels)

    def forward(self, x, embedding):
        gamma = self.gamma(embedding).unsqueeze(-1)
        beta = self.beta(embedding).unsqueeze(-1)
        return gamma * x + beta

class SpeakerConditionedEnhancer(torch.nn.Module):
    def __init__(self, embedding_dim=256):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv1d(1, 64, 15, padding=7),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 128, 15, padding=7),
            torch.nn.ReLU()
        )
        self.film = FiLM(128, embedding_dim)
        self.decoder = torch.nn.Sequential(
            torch.nn.Conv1d(128, 64, 15, padding=7),
            torch.nn.ReLU(),
            torch.nn.Conv1d(64, 1, 15, padding=7),
        )

    def forward(self, x, embedding):
        x = self.encoder(x)
        x = self.film(x, embedding)
        return self.decoder(x)

enhancer_model = SpeakerConditionedEnhancer()

In [ ]:
# ✅ STEP 4: Load Reference Speaker Embeddings
# Example: npy files per speaker
import glob
from collections import defaultdict

def build_speaker_db(clean_root="/content/drive/MyDrive/programmingAssignmentShreya/clean_refs_trial", max_utts=5):
    speaker_db = {}
    for speaker_dir in os.listdir(clean_root):
        speaker_path = os.path.join(clean_root, speaker_dir)
        if not os.path.isdir(speaker_path):
            continue

        embeddings = []
        utt_files = glob.glob(os.path.join(speaker_path, "*.wav"))[:max_utts]
        for utt in utt_files:
            audio, sr = torchaudio.load(utt)
            audio = audio.squeeze().numpy()
            emb = get_embedding(audio, sr)
            embeddings.append(emb)

        if embeddings:
            avg_embedding = np.mean(embeddings, axis=0)
            speaker_db[speaker_dir] = avg_embedding
            print(f"Loaded speaker: {speaker_dir}, used {len(embeddings)} utterances.")
    return speaker_db

# 🏗️ Build it once
speaker_db = build_speaker_db("/content/drive/MyDrive/programmingAssignmentShreya/clean_refs_trial")


In [ ]:
# ============================================
# 🔊 Load and Pad Audio
# ============================================
def get_embedding(audio, sr):
    min_length = 16000  # Ensure at least 1 second of audio
    if len(audio) < min_length:
        pad_length = min_length - len(audio)
        audio = np.pad(audio, (0, pad_length), mode='constant')
        print(f"[Info] Padded audio from {len(audio)-pad_length} to {len(audio)} samples.")

    inputs = processor(audio, sampling_rate=sr, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs).last_hidden_state  # (batch_size, seq_len, hidden_size)
        pooled = outputs.mean(dim=1)
    return pooled.squeeze().cpu().numpy()

# ============================================
# 🔍 Identify Speaker
# ============================================
def identify_speaker(embedding):
    max_sim = -1
    best_id = "unknown"
    for speaker_id, ref_emb in speaker_db.items():
        sim = cosine_similarity(embedding.reshape(1, -1), ref_emb.reshape(1, -1))[0][0]
        if sim > max_sim:
            best_id, max_sim = speaker_id, sim
    return best_id, max_sim

# ============================================
# 🔁 Process Mixed Audio File
# ============================================
def process_audio(mix_path, output_dir):
    # Step 1: Separate using SepFormer
    est_sources = sepformer.separate_file(path=mix_path)
    sr = 16000  # SepFormer outputs at 16kHz

    for i, stream in enumerate(est_sources):
        stream_np = stream.numpy()  # Convert to NumPy array

        # Step 2: Extract speaker embedding
        emb = get_embedding(stream_np, sr)
        speaker_id, sim = identify_speaker(emb)
        print(f"Stream {i}: Speaker {speaker_id} (sim={sim:.2f})")

        # Step 3: Prepare input for enhancer
        enhancer_input = torch.tensor(stream_np).unsqueeze(0).unsqueeze(0)  # [1, 1, T]

        # OPTIONAL: Resample to 8kHz if enhancer model was trained on 8kHz audio
        expected_sr = expected_sr = 16000
  # <--- Change to 16000 if enhancer uses 16kHz
        if sr != expected_sr:
            print(f"Resampling the audio from {sr} Hz to {expected_sr} Hz")
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=expected_sr)
            enhancer_input = resampler(enhancer_input.squeeze(0)).unsqueeze(0)  # [1, 1, T]
            sr = expected_sr

        # Step 3b: Pad enhancer input if too short
        min_len = 15  # Safe minimum for Conv1D
        input_len = enhancer_input.shape[-1]
        if input_len < min_len:
            pad_len = min_len - input_len
            enhancer_input = torch.nn.functional.pad(enhancer_input, (0, pad_len))
            print(f"[Enhancer] Padded enhancer input from {input_len} to {enhancer_input.shape[-1]}")

        # Step 4: Run enhancer model
        embedding_tensor = torch.tensor(emb).unsqueeze(0)
        with torch.no_grad():
            enhanced = enhancer_model(enhancer_input, embedding_tensor).squeeze().cpu().numpy()

        # Step 5: Save enhanced output
        filename = f"{os.path.basename(mix_path).split('.')[0]}_speaker_{speaker_id}_stream{i}.wav"
        out_path = os.path.join(output_dir, filename)
        sf.write(out_path, enhanced, sr)
        print(f"[Saved] {out_path}")


In [ ]:
# ✅ STEP 6: Run It!
os.makedirs("/content/drive/MyDrive/programmingAssignmentShreya/enhanced_outputs", exist_ok=True)

# Replace this with your mixed test set folder
test_files = glob.glob("/content/drive/MyDrive/programmingAssignmentShreya/mixed_test_trial_audio/*.wav")

for path in test_files:
    print(f"Processing: {path}")
    process_audio(path, "/content/drive/MyDrive/programmingAssignmentShreya/enhanced_outputs")


Processing: /content/drive/MyDrive/programmingAssignmentShreya/mixed_test_trial_audio/mix2450.wav


TypeError: process_audio() missing 6 required positional arguments: 'speaker_db', 'sepformer', 'processor', 'device', 'model', and 'enhancer_model'